# ** Making the df with each ind as a row** 

### ** 1) Copy the informative portion of the provided data and pasted into a csv file "Book1.csv** 

###  ** 2) removed the "ALL" rows**

In [ ]:
!grep -v "ALL" Book1.csv > Data_noAll.csv

### ** 3) created a dictionary where each entry is an individual ** 

In [1]:
import pandas as pd

In [4]:
raw_lines = open("Data_noAll.csv").readlines()

In [8]:
len(raw_lines)

727

In [157]:
dict_ind_rows = {} # dictionary to fill 
row_n = 0   #total number of rows 
total_people = 0 # total number of patients in study 
list_total_sum_off = [] #a collection of mess ups: caution measure 

for i in range(1, len(raw_lines), 3): #iterate through the lines in 3s as 3 rows belong to each factor set(tx, event, censor)
    
    prov = raw_lines[i].split(",")[0]
    con_act = raw_lines[i].split(",")[1]
    sex = raw_lines[i].split(",")[2]
    age = raw_lines[i].split(",")[3]
    
    #note that everyone in a given factor set group either 
    #1) dropped from event, 
    #2) dropped from census or 
    #3) stayed the whole time 
    #so only need to look at the values in the census and event rows to see whats happening 
    
    tx_list = raw_lines[i].split(",")[5:-1]
    events_list = raw_lines[i+1].split(",")[5:-1] # getting the last 39 elements of the evtns row, and excluding the "\n" at the end
    censor_list = raw_lines[i+2].split(",")[5:-1] # getting the last 39 elements of the censor row, and excluding the "\n" at the end 
    n_censor = 0 #nmber of people in this factor set that dropped from censor
    n_event = 0 #nmber of people in this factor set that dropped from event
    
    #1) adding people to dict who dropped from event (going through the event list, blanks and numbers)
    for index, element in enumerate(events_list): 
        try: 
            int(element)
        except: 
            None
        else: 
            reason_for_drop = "event"
            month_dropped = index # the index of the list is equivalent to how many months they stay in the program 
            for j in range(0, int(element)):
                dict_ind_rows[row_n] = [prov, con_act, sex, age, reason_for_drop, month_dropped]
                n_event += 1
                row_n += 1
    #2) adding people to dict who dropped from censor (going through the censor list, blanks and numbers)        
    for index, element in enumerate(censor_list): 
        try: 
            int(element)
        except: 
            None
        else: 
            reason_for_drop = "censor"
            month_dropped = index # the index of the list is equivalent to how many months they stay in the program 
            for j in range(0, int(element)): 
                dict_ind_rows[row_n] = [prov, con_act, sex, age, reason_for_drop, month_dropped]
                n_censor += 1
                row_n += 1
    
    #3) adding those who lasted the whole duration (no event or censor value associated, this row has Tx values until the end: M39) 
    total_atEnd = tx_list[-1]
    n_noDrop = 0
    try: 
        int(total_atEnd)
    except: 
        None
    else: 
        reason_for_drop = "NA"
        month_dropped = "NA"
        n_noDrop = int(total_atEnd)
        for k in range(0,n_noDrop): 
            dict_ind_rows[row_n] = [prov, con_act, sex, age, reason_for_drop, month_dropped]
            row_n += 1
    
    #check to make sure the totals for this factor set group make sense 
    total_inGroup = int(raw_lines[i].split(",")[5]) #total number of people in this facotr set group 
    total_people += total_inGroup
    if total_inGroup - n_noDrop != n_event + n_censor: 
        print("oh no error")
        list_total_sum_off.append([total_inGroup, [prov, con_act, sex, age,total_inGroup], tx_list, n_event, events_list, n_censor, censor_list])


### ** 4) convert the dictionary to df and save as csv **

In [164]:
ind_df = pd.DataFrame.from_dict(dict_ind_rows, orient = "index")

In [165]:
ind_df.head()

,0,1,2,3,4,5
0,AB,No,F,18-19,event,12
1,AB,No,F,25-29,censor,29
2,AB,No,F,30-34,event,5
3,AB,No,F,30-34,censor,4
4,AB,No,F,35-39,event,1


In [175]:
ind_df = pd.DataFrame.from_dict(dict_ind_rows, orient = "index")
ind_df.columns = ["Prov", "Con_ACT", "Sex", "Age", "Reason_for_drop", "Month_lasted"]


In [182]:
print("number of total patients", len(ind_df))
print("number of censored patients", len(ind_df[ind_df.Reason_for_drop == "censor"]))
print("number of event patients", len(ind_df[ind_df.Reason_for_drop == "event"]))

number of total patients 1441
number of censored patients 820
number of event patients 617


In [179]:
pd.DataFrame.to_csv(ind_df, "Dataframe_with_ind_rows.csv")